In [16]:
import jsonlines
from langchain.schema import Document
from typing import Iterable

In [17]:
from langchain_unstructured import UnstructuredLoader
from unstructured_client import UnstructuredClient

In [18]:
import json

# Load the JSON file into a dictionary
with open("OldDominion_metadata1.json", "r") as file:
    pdf_files = json.load(file)

In [19]:
print(pdf_files)
# Fetch the keys from the dictionary
file_paths = list(pdf_files.keys())

print(file_paths)


{'Old_Dominion\\Chain_Bridge_Partners\\2021.07.02 - Chain Bridge Partners - Deed of Lease.pdf': {'file_name': 'Chain Bridge Partners - Deed of Lease', 'building_name': 'Old Dominion', 'tenant_name': 'Chain Bridge Partners', 'date': '2021.07.02'}, 'Old_Dominion\\Chain_Bridge_Partners\\Chain Bridge Partners - Commencement.pdf': {'file_name': 'Chain Bridge Partners - Commencement.pdf', 'building_name': 'Old Dominion', 'tenant_name': 'Chain Bridge Partners', 'date': ''}, 'Old_Dominion\\Collis_&_Associates\\2015.01.07 - Collis & Associates - Original Lease dtd.pdf': {'file_name': 'Collis & Associates - Original Lease dtd', 'building_name': 'Old Dominion', 'tenant_name': 'Collis & Associates', 'date': '2015.01.07'}, 'Old_Dominion\\Collis_&_Associates\\2015.03.30 - Collis & Associates - Exhibit B dtd.pdf': {'file_name': 'Collis & Associates - Exhibit B dtd', 'building_name': 'Old Dominion', 'tenant_name': 'Collis & Associates', 'date': '2015.03.30'}, 'Old_Dominion\\Collis_&_Associates\\2017.0

In [20]:
client = UnstructuredClient(
    api_key_auth="ELlpsEOVuPLb9hjEbfUSxNm3DxnNWE",
    server="free-api"
)


In [21]:
import os
from PyPDF2 import PdfReader

def check_pdf_files(file_paths):
    bad_files = []
    for file_path in file_paths:
        try:
            # Attempt to open the PDF
            with open(file_path, 'rb') as f:
                reader = PdfReader(f)
                # Try to read the first page
                _ = reader.pages[0]
            print(f"{file_path}: OK")
        except Exception as e:
            # Catch any errors related to reading the PDF
            print(f"{file_path}: BAD - {str(e)}")
            bad_files.append(file_path)
    return bad_files

bad_files = check_pdf_files(file_paths)

print("\nBad Files:")
for bad_file in bad_files:
    print(bad_file)


Old_Dominion\Chain_Bridge_Partners\2021.07.02 - Chain Bridge Partners - Deed of Lease.pdf: OK
Old_Dominion\Chain_Bridge_Partners\Chain Bridge Partners - Commencement.pdf: OK
Old_Dominion\Collis_&_Associates\2015.01.07 - Collis & Associates - Original Lease dtd.pdf: OK
Old_Dominion\Collis_&_Associates\2015.03.30 - Collis & Associates - Exhibit B dtd.pdf: OK
Old_Dominion\Collis_&_Associates\2017.04.03 - Collis & Associates - 1st Amendment dtd.pdf: OK
Old_Dominion\Collis_&_Associates\2019.09.05 - Collis & Associates - 2nd Amendment.pdf: OK
Old_Dominion\Collis_&_Associates\Collis & Associates - Commencement_2nd Amendment.pdf: OK
Old_Dominion\Dell_Management\Dell Management - Storage License Agreement.pdf: OK
Old_Dominion\Dell_Management\Dell Management - Third Ammendment to the Lease.pdf: OK
Old_Dominion\General_Counsel\2014.01.28 - General Counsel - Original lease dtd.pdf: OK
Old_Dominion\General_Counsel\2014.03.13 - General Counsel - Exhibit B dtd.pdf: OK
Old_Dominion\General_Counsel\201

In [22]:
loader = UnstructuredLoader(
    file_path = file_paths,
    partition_via_api=True,
    client=client,
    strategy="hi_res",
    hi_res_model_name="yolox",
    chunking_strategy="by_title",
    pdf_infer_table_structure=True,
    max_characters=3700,
)

docs = loader.load()

INFO: Preparing to split document for partition.
INFO: Concurrency level set to 5
INFO: Splitting pages 1 to 35 (35 total)
INFO: Determined optimal split size of 7 pages.
INFO: Partitioning 5 files with 7 page(s) each.
INFO: Partitioning set #1 (pages 1-7).
INFO: Partitioning set #2 (pages 8-14).
INFO: Partitioning set #3 (pages 15-21).
INFO: Partitioning set #4 (pages 22-28).
INFO: Partitioning set #5 (pages 29-35).
INFO: HTTP Request: POST https://api.unstructured.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructured.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructured.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructured.io/general/v0/general "HTTP/1.1 200 OK"
INFO: Successfully partitioned set #1, elements added to the final result.
INFO: Successfully partitioned set #2, elements added to the final result.
INFO: Successfully partitioned set #3, elements added to the final re

SDKError: API error occurred: Status 402
{"detail":"Insufficient API quota for owner organization"}

In [8]:
def save_docs_to_jsonl(documents: Iterable[Document], file_path: str) -> None:
    with jsonlines.open(file_path, mode="w") as writer:
        for doc in documents:
            writer.write(doc.dict())

def load_docs_from_jsonl(file_path: str) -> Iterable[Document]:
    documents = []
    with jsonlines.open(file_path, mode="r") as reader:
        for doc in reader:
            documents.append(Document(**doc))
    return documents


save_docs_to_jsonl(docs, "./docs_beverly.json")
loaded_docs = load_docs_from_jsonl("./docs_beverly.json")

NameError: name 'docs' is not defined

In [ ]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

pretty_print_docs(loaded_docs[:2])

In [ ]:
import os
from datetime import datetime
loaded_docs = load_docs_from_jsonl("./docs.json")
def extract_file_details(file_path):
    # Extract the file name from the full path
    file_name = os.path.basename(file_path)
    
    # Split the file name by '-' to get the individual parts
    parts = file_name.split('-')
    
    # Determine if the date is present in the file name
    if len(parts) >= 3:
        date_str = parts[0].strip()
        tenant_name = parts[1].strip()
        file_name = '-'.join(parts[2:]).strip('.pdf').strip()
    else:
        tenant_name = parts[0].strip()
        file_name = '-'.join(parts[1:]).strip('.pdf').strip()
        date_str = ""
    
    # Extract the building name from the file path
    building_name = file_path.split('\\')[1]
    
    return {
        'building_name': building_name,
        'tenant_name': tenant_name,
        'date': date_str,
        'filename': file_name
    }

def append_metadata(doc):
    metadata = extract_file_details(doc.metadata['filename'])
    doc.metadata['building_name'] = metadata['building_name']
    doc.metadata['tenant_name'] = metadata['tenant_name']
    doc.metadata['date'] = metadata['date']
    doc.metadata['filename'] = metadata['filename']
    return doc

for i, doc in enumerate(loaded_docs):
    print("DFILE: ", doc.metadata['filename'], "i: ", i)
    with open("temp.md", "+a") as f:
        f.write(f"DFILELE: {doc.metadata['filename']}")
    append_metadata(doc)